<a href="https://colab.research.google.com/github/indhu68/Intro_To_DL/blob/main/Intro_to_DL_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

                                    INTRODUCTION TO DEEP LEARNING

                                            Homework 4

Name: Indhuja Gudluru

Student ID: 801366046

PROBLEM 1:

English To French Translation(Without Attention)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random
from itertools import chain

In [10]:
# Sample dataset of English-French sentence pairs
english_to_french = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur")
]


In [12]:
# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 12

char_to_index_Eng = word_to_index = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_Eng = {i: char for char, i in char_to_index_Eng.items()}

char_to_index_French = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_French = {i: char for char, i in char_to_index_French.items()}

class EnglishFrenchDataset(Dataset):
    """Custom Dataset class for handling English-French pairs."""
    def __init__(self, dataset, char_to_index_Eng, char_to_index_French):
        self.dataset = dataset
        self.char_to_index_Eng = char_to_index_Eng
        self.char_to_index_French = char_to_index_French

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        English_sentence, French_sentence = self.dataset[idx]
        English_tensor = torch.tensor([self.char_to_index_Eng[word] for word in English_sentence.split()] + [EOS_token], dtype=torch.long)
        French_tensor = torch.tensor([self.char_to_index_French[word] for word in French_sentence.split()] + [EOS_token], dtype=torch.long)
        return English_tensor, French_tensor

English_French_dataset = EnglishFrenchDataset(english_to_french, char_to_index_Eng, char_to_index_French)
dataloader = DataLoader(English_French_dataset, batch_size=1, shuffle=True)

input_size=len(char_to_index_Eng)
hidden_size=256
output_size=len(char_to_index_French)

# Setting the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Encoder(nn.Module):
    """Encoder that processes the input sequence and returns its hidden states."""
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

# Decoder without attention mechanism
class Decoder(nn.Module):
    """Decoder without attention mechanism."""
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))


learning_rate = 0.01
criterion = nn.NLLLoss()
encoder_English = Encoder(input_size=input_size, hidden_size=hidden_size).to(device)
decoder_French = Decoder(hidden_size=hidden_size, output_size=output_size).to(device)
encoder_optimizer = optim.SGD(encoder_English.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder_French.parameters(), lr=learning_rate)

# Training loop
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
        #Calculate Loss
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

n_epochs = 200

for epoch in range(n_epochs):
    total_loss = 0
    for input_tensor_English, target_tensor_French in dataloader:
        input_tensor_English = input_tensor_English[0].to(device)
        target_tensor_French = target_tensor_French[0].to(device)

        loss = train(input_tensor_English, target_tensor_French, encoder_English, decoder_French, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    if (epoch+10) % 10 == 0:
        print(f'Epoch {epoch+10}, Loss: {total_loss / len(dataloader)}')


def evaluate_and_show_examples(encoder, decoder, dataloader, criterion, n_examples=5):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_English, target_tensor_French) in enumerate(dataloader):
            input_tensor_English = input_tensor_English[0].to(device)
            target_tensor_French = target_tensor_French[0].to(device)

            encoder_hidden = encoder.initHidden()


            input_length = input_tensor_English.size(0)
            target_length = target_tensor_French.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor_English[ei].unsqueeze(0), encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor_French[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:
                    break
            # Calculate and print loss and accuracy for the evaluation
            total_loss += loss.item() / target_length
            total_sentences += 1

            if predicted_indices == target_tensor_French.tolist():
                correct_predictions += 1
            # Optionally, print some examples
            if i < n_examples:
                predicted_string = ' '.join([index_to_char_French[index] for index in predicted_indices if index not in (SOS_token, EOS_token)])
                target_string = ' '.join([index_to_char_French[index.item()] for index in target_tensor_French if index.item() not in (SOS_token, EOS_token)])
                input_string = ' '.join([index_to_char_Eng[index.item()] for index in input_tensor_English if index.item() not in (SOS_token, EOS_token)])

                print(f'Input: {input_string}, Target: {target_string}, Predicted: {predicted_string}')
        # Print overall evaluation results
        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss}, Accuracy: {accuracy}')

# Perform evaluation
evaluate_and_show_examples(encoder_English, decoder_French, dataloader, criterion)


Epoch 10, Loss: 3.3776268181322133
Epoch 20, Loss: 2.8885273264993265
Epoch 30, Loss: 2.402769331872106
Epoch 40, Loss: 1.6851317312515441
Epoch 50, Loss: 0.8612876169633872
Epoch 60, Loss: 0.27985390731567594
Epoch 70, Loss: 0.10893945201764493
Epoch 80, Loss: 0.05843526253960927
Epoch 90, Loss: 0.03587743057330854
Epoch 100, Loss: 0.02705417489204836
Epoch 110, Loss: 0.021724904674427694
Epoch 120, Loss: 0.01813294672484195
Epoch 130, Loss: 0.015542850619743745
Epoch 140, Loss: 0.013585661279186962
Epoch 150, Loss: 0.012057136579018708
Epoch 160, Loss: 0.010828104893072378
Epoch 170, Loss: 0.009823302135360664
Epoch 180, Loss: 0.008982353351996429
Epoch 190, Loss: 0.00827191506593673
Epoch 200, Loss: 0.007662420812975042
Input: The flowers bloom in spring, Target: Les fleurs fleurissent au printemps, Predicted: Les fleurs fleurissent au printemps
Input: She sings a song, Target: Elle chante une chanson, Predicted: Elle chante une chanson
Input: We eat breakfast together, Target: Nous

PROBLEM 2: With Attention

In [16]:
# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 12

char_to_index_Eng = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_Eng = {i: char for char, i in char_to_index_Eng.items()}

char_to_index_French = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_French = {i: char for char, i in char_to_index_French.items()}

class EnglishFrenchDataset(Dataset):
    """Custom Dataset class for handling English-French sentence pairs."""
    def __init__(self, dataset, char_to_index_Eng, char_to_index_French):
        self.dataset = dataset
        self.char_to_index_Eng = char_to_index_Eng
        self.char_to_index_French = char_to_index_French

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        English_sentence, French_sentence = self.dataset[idx]
        English_tensor = torch.tensor([self.char_to_index_Eng[word] for word in English_sentence.split()] + [EOS_token], dtype=torch.long)
        French_tensor = torch.tensor([self.char_to_index_French[word] for word in French_sentence.split()] + [EOS_token], dtype=torch.long)
        return English_tensor, French_tensor

English_French_dataset = EnglishFrenchDataset(english_to_french, char_to_index_Eng, char_to_index_French)
dataloader = DataLoader(English_French_dataset, batch_size=1, shuffle=True)

input_size_English = len(char_to_index_Eng)
input_size_French = len(char_to_index_French)
hidden_size = 256

# Setting the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize encoder and decoder
class Encoder(nn.Module):
    """Encoder that processes the input sequence and returns its hidden states."""
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

class AttnDecoder(nn.Module):
    """Decoder with attention mechanism."""
    def __init__(self, hidden_size, output_size, max_length=12, dropout_p=0.1):
        super(AttnDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = torch.softmax(
            self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = torch.relu(output)
        output, hidden = self.lstm(output, hidden)

        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))
# Define loss function and optimizer
learning_rate = 0.01
criterion = nn.NLLLoss()
encoder_English = Encoder(input_size=input_size_English, hidden_size=hidden_size).to(device)
decoder_French = AttnDecoder(hidden_size=hidden_size, output_size=input_size_French).to(device)
encoder_optimizer = optim.SGD(encoder_English.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder_French.parameters(), lr=learning_rate)

# Training loop
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
        #Calculate Loss
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

n_epochs = 200

for epoch in range(n_epochs):
    total_loss = 0
    for input_tensor_English, target_tensor_French in dataloader:
        input_tensor_English = input_tensor_English[0].to(device)
        target_tensor_French = target_tensor_French[0].to(device)
        # Perform a single training step and update total loss
        loss = train(input_tensor_English, target_tensor_French, encoder_English, decoder_French, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    if (epoch+10) % 10 == 0:
        print(f'Epoch {epoch+10}, Loss: {total_loss / len(dataloader)}')

def evaluate_and_show_examples(encoder, decoder, dataloader, criterion, n_examples=5):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_English, target_tensor_French) in enumerate(dataloader):
            input_tensor_English = input_tensor_English[0].to(device)
            target_tensor_French = target_tensor_French[0].to(device)

            encoder_hidden = encoder.initHidden()
            input_length = input_tensor_English.size(0)
            target_length = target_tensor_French.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor_English[ei].unsqueeze(0), encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor_French[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:
                    break

            total_loss += loss.item() / target_length
            total_sentences += 1

            if predicted_indices == target_tensor_French.tolist():
                correct_predictions += 1

            if i < n_examples:
                predicted_string = ' '.join([index_to_char_French[index] for index in predicted_indices if index not in (SOS_token, EOS_token)])
                target_string = ' '.join([index_to_char_French[index.item()] for index in target_tensor_French if index.item() not in (SOS_token, EOS_token)])
                input_string = ' '.join([index_to_char_Eng[index.item()] for index in input_tensor_English if index.item() not in (SOS_token, EOS_token)])

                print(f'Input: {input_string}, Target: {target_string}, Predicted: {predicted_string}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss}, Accuracy: {accuracy}')

# Perform evaluation
evaluate_and_show_examples(encoder_English, decoder_French, dataloader, criterion)




Epoch 10, Loss: 3.8849162718065746
Epoch 20, Loss: 2.929303123345942
Epoch 30, Loss: 2.13454469038614
Epoch 40, Loss: 1.170867978300269
Epoch 50, Loss: 0.33202017654870836
Epoch 60, Loss: 0.13107812933038615
Epoch 70, Loss: 0.058844656905401645
Epoch 80, Loss: 0.038584630034205064
Epoch 90, Loss: 0.028168036062727327
Epoch 100, Loss: 0.022362104958102605
Epoch 110, Loss: 0.018229276137635766
Epoch 120, Loss: 0.015434511409278777
Epoch 130, Loss: 0.013259936095960474
Epoch 140, Loss: 0.011784561283228304
Epoch 150, Loss: 0.01061497910750346
Epoch 160, Loss: 0.00951341514336838
Epoch 170, Loss: 0.008610720349423139
Epoch 180, Loss: 0.007909056553960778
Epoch 190, Loss: 0.007340013164715113
Epoch 200, Loss: 0.0068235557258529995
Input: The baby cries, Target: Le bébé pleure, Predicted: Le bébé pleure
Input: We eat breakfast together, Target: Nous prenons le petit déjeuner ensemble, Predicted: Nous prenons le petit déjeuner ensemble
Input: She runs in the park, Target: Elle court dans le p

Problem 3: French to English Transl



In [17]:
french_to_english = [(french, english) for (english, french) in english_to_french]

# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 12

char_to_index_Eng = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_Eng = {i: char for char, i in char_to_index_Eng.items()}

char_to_index_French = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_French = {i: char for char, i in char_to_index_French.items()}

class EnglishFrenchDataset(Dataset):
    """Custom Dataset class for handling French - English sentence pairs."""
    def __init__(self, dataset, char_to_index_Eng, char_to_index_French):
        self.dataset = dataset
        self.char_to_index_Eng = char_to_index_Eng
        self.char_to_index_French = char_to_index_French

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        English_sentence, French_sentence = self.dataset[idx]
        English_tensor = torch.tensor([self.char_to_index_Eng[word] for word in English_sentence.split()] + [EOS_token], dtype=torch.long)
        French_tensor = torch.tensor([self.char_to_index_French[word] for word in French_sentence.split()] + [EOS_token], dtype=torch.long)
        return English_tensor, French_tensor

English_French_dataset = EnglishFrenchDataset(french_to_english, char_to_index_French, char_to_index_Eng)
dataloader = DataLoader(English_French_dataset, batch_size=1, shuffle=True)

input_size_English = len(char_to_index_Eng)
input_size_French = len(char_to_index_French)
hidden_size = 256

class Encoder(nn.Module):
    """Encoder that processes the input sequence and returns its hidden states."""
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

# Decoder without attention mechanism
class Decoder(nn.Module):
    """Decoder without attention mechanism."""
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


learning_rate = 0.01
criterion = nn.NLLLoss()
encoder_English = Encoder(input_size=input_size, hidden_size=hidden_size).to(device)
decoder_French = Decoder(hidden_size=hidden_size, output_size=output_size).to(device)
encoder_optimizer = optim.SGD(encoder_English.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder_French.parameters(), lr=learning_rate)

# Training loop
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

n_epochs = 200

for epoch in range(n_epochs):
    total_loss = 0
    for input_tensor_English, target_tensor_French in dataloader:
        input_tensor_English = input_tensor_English[0].to(device)
        target_tensor_French = target_tensor_French[0].to(device)

        loss = train(input_tensor_English, target_tensor_French, encoder_English, decoder_French, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    if (epoch+10) % 10 == 0:
        print(f'Epoch {epoch+10}, Loss: {total_loss / len(dataloader)}')

def evaluate_and_show_examples(encoder, decoder, dataloader, criterion, n_examples=5):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_English, target_tensor_French) in enumerate(dataloader):
            input_tensor_English = input_tensor_English[0].to(device)
            target_tensor_French = target_tensor_French[0].to(device)

            encoder_hidden = encoder.initHidden()
            input_length = input_tensor_English.size(0)
            target_length = target_tensor_French.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor_English[ei].unsqueeze(0), encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor_French[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:
                    break

            total_loss += loss.item() / target_length
            total_sentences += 1

            if predicted_indices == target_tensor_French.tolist():
                correct_predictions += 1

            if i < n_examples:
                predicted_string = ' '.join([index_to_char_French[index] for index in predicted_indices if index not in (SOS_token, EOS_token)])
                target_string = ' '.join([index_to_char_French[index.item()] for index in target_tensor_French if index.item() not in (SOS_token, EOS_token)])
                input_string = ' '.join([index_to_char_Eng[index.item()] for index in input_tensor_English if index.item() not in (SOS_token, EOS_token)])

                print(f'Input: {input_string}, Target: {target_string}, Predicted: {predicted_string}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss}, Accuracy: {accuracy}')

# Perform evaluation
evaluate_and_show_examples(encoder_English, decoder_French, dataloader, criterion)

Epoch 10, Loss: 4.227311626450841
Epoch 20, Loss: 3.2972034569647706
Epoch 30, Loss: 2.2716345999958927
Epoch 40, Loss: 1.6651913511804683
Epoch 50, Loss: 0.7036361116720118
Epoch 60, Loss: 0.19447064117362045
Epoch 70, Loss: 0.0805016780929869
Epoch 80, Loss: 0.04634820142957852
Epoch 90, Loss: 0.032974023854356535
Epoch 100, Loss: 0.0255248185942939
Epoch 110, Loss: 0.020763189409990217
Epoch 120, Loss: 0.017462355977105156
Epoch 130, Loss: 0.01503640206296851
Epoch 140, Loss: 0.013187654784505629
Epoch 150, Loss: 0.011733200802797786
Epoch 160, Loss: 0.010557303634623668
Epoch 170, Loss: 0.009588584294352605
Epoch 180, Loss: 0.00877827821828162
Epoch 190, Loss: 0.008090222046820559
Epoch 200, Loss: 0.007499800812241078
Input: Elle parle français couramment, Target: She speaks French fluently, Predicted: She speaks French fluently
Input: Il peint des paysages, Target: He paints landscapes, Predicted: He paints landscapes
Input: J'ai froid, Target: I am cold, Predicted: I am cold
Inpu